In [1]:
import os
import sys
sys.path.append("/home/rohit/PhD_Work/GM_my_version/Graph_matching/")
from sklearn.cluster import KMeans
import networkx as nx
import numpy as np
from graph_generation.load_graphs_and_create_metadata import dataset_metadata
from graph_matching_tools.metrics import matching
from graph_matching_tools.io.graph_dataset import GraphDataset
from graph_matching_tools import *
import matplotlib.pyplot as plt
import tools.graph_processing as gp
import scipy.io as sio
import graph_matching_tools.algorithms.kernels.gaussian as gaussian
import graph_matching_tools.algorithms.multiway.mkergm as mkergm
import graph_matching_tools.algorithms.multiway.matcheig as matcheig
import graph_matching_tools.algorithms.multiway.msync as msync
import graph_matching_tools.algorithms.kernels.linear as lin_kern
import graph_matching_tools.utils.permutations as permut
import graph_matching_tools.algorithms.kernels.utils as kutils
import graph_matching_tools.utils.utils as utils
import graph_matching_tools.algorithms.kernels.rff as rff
#import graph_matching_tools.metrics.matching as metrics
from graph_matching_tools.metrics import matching

In [2]:
def get_permutation_matrix_from_dictionary(matching, g_sizes):
    """
    Create the full permutation matrix from the matching result
    :param matching: the matching result for each graph (nodes number, assignment)
    :param g_sizes: the list of the size of the different graph
    :return: the full permutation matrix
    """
    f_size = int(np.sum(g_sizes))
    res = np.zeros((f_size, f_size))

    idx1 = 0
    for i_g1 in range(len(g_sizes)):
        idx2 = 0
        for i_g2 in range(len(g_sizes)):
            match = matching["{},{}".format(i_g1, i_g2)]
            for k in match:
                res[idx1 + int(k), idx2 + match[k]] = 1
            idx2 += g_sizes[i_g2]
        idx1 += g_sizes[i_g1]
        
    np.fill_diagonal(res,1)
    return res

In [3]:
path_to_graph_folder = '../data/simu_graph/test_small_set//0/noise_1000,outliers_varied/graphs/'
path_to_groundtruth_ref = '../data/simu_graph/test_small_set/0/noise_1000,outliers_varied/permutation_to_ref_graph.gpickle'
path_to_groundtruth = '../data/simu_graph/test_small_set/0/noise_1000,outliers_varied/ground_truth.gpickle'
path_to_graph_folder_dummy = '../data/simu_graph/test_small_set_dummy//0/noise_1000,outliers_varied/0/graphs/'

In [4]:
graph_meta = GraphDataset(path_to_graph_folder, path_to_groundtruth_ref)

In [5]:
#list_graphs_dummy = gp.load_graphs_in_list(path_to_graph_folder_dummy)

In [6]:
all_graphs = graph_meta.list_graphs
g_sizes = graph_meta.sizes
#g_dummy_sizes = [nx.number_of_nodes(g) for g in list_graphs_dummy]
full_size = int(np.sum(g_sizes))

In [7]:
node_kernel = gaussian.create_gaussian_node_kernel(10, "coord")
knode = kutils.create_full_node_affinity_matrix(all_graphs, node_kernel)

vectors, offsets = rff.create_random_vectors(1, 100, 0.15)
full_size = knode.shape[0]
phi = np.zeros((100, full_size, full_size))
index = 0
for i in range(len(all_graphs)):
    g_phi = rff.compute_phi(all_graphs[i], "geodesic_distance", vectors, offsets)
    phi[:, index : index + g_sizes[i], index : index + g_sizes[i]] = g_phi
    index += g_sizes[i]

In [33]:
data = 10.0
np.sqrt(2 / vectors.shape[1]) * np.cos(np.dot(data.T, vectors) + offsets)

AttributeError: 'float' object has no attribute 'T'

In [ ]:
# phi = np.zeros((1, full_size, full_size)) # data_dimension 
# index = 0

# for i in range(len(graph_meta.list_graphs)):
#     g_phi = np.zeros((1, g_sizes[i] , g_sizes[i])) 
#     for u, v, data in graph_meta.list_graphs[i].edges.data("geodesic_distance"):
#         g_phi[:, u, v] = data
#         g_phi[:, v, u] = data
#     phi[:, index:index + g_sizes[i], index:index + g_sizes[i]] = g_phi
#     index += g_sizes[i]

In [13]:
graph_meta.list_graphs[0].nodes.data()[0]

{'coord': array([ 51.51622912, -83.93188167, -17.36425571]),
 'is_dummy': False,
 'is_outlier': False}

In [14]:
x_init = np.ones(knode.shape) / knode.shape[0]
#x_init = res_kmeans

In [15]:
norm_knode = np.median(g_sizes)
knode /= norm_knode

In [16]:
gradient = mkergm.create_gradient(phi, knode)

In [17]:
rank = max(graph_meta.sizes)
m_res = mkergm.mkergm(gradient, g_sizes, rank, iterations=50, init = x_init, projection_method = 'matcheig')

In [ ]:
# Scores

In [18]:
ground_truth =  nx.read_gpickle(path_to_groundtruth)
res = get_permutation_matrix_from_dictionary(ground_truth, graph_meta.sizes)
print('res shape : ',res.shape)

res shape :  (804, 804)


In [19]:
# # Remove dummy nodes from X matrix

X_mkergm = m_res

# dummy_mask = [list(nx.get_node_attributes(graph,'is_dummy').values()) for graph in list_graphs_dummy]
# dummy_mask = sum(dummy_mask,[])
# dummy_indexes = [i for i in range(len(dummy_mask)) if dummy_mask[i]==True]   

# X_mkergm = np.delete(X_mkergm,dummy_indexes,0) # delete the dummy rows
# X_mkergm = np.delete(X_mkergm,dummy_indexes,1) # delete the dummy columns


In [20]:
X_mkergm.shape

(804, 804)

In [21]:
f1, prec, rec = matching.compute_f1score(X_mkergm,res)
f1,prec,rec

(0.7135048231511254, 0.6320136713187127, 0.8191214470284238)

In [28]:
m_res_matcheig = matcheig.matcheig(knode, rank, g_sizes)

In [29]:
f1, prec, rec = matching.compute_f1score(m_res_matcheig,res)
f1,prec,rec

(0.8495176848874598, 0.7524921674736542, 0.9752676264304171)

In [ ]:
u_nodes = msync.msync(knode, g_sizes, rank, lambda x: 0)
m_res_msync = u_nodes @ u_nodes.T

In [ ]:
f1, prec, rec = matching.compute_f1score(m_res_msync,res)
f1,prec,rec

### Test with Kmeans

In [22]:
def get_all_coords(list_graphs):
    all_coords = []
    for g in list_graphs:
        coords = np.array(list(nx.get_node_attributes(g,'coord').values()))
        all_coords.extend(coords)
    all_coords = np.array(all_coords)
    
    return all_coords

In [23]:
def create_perm_from_labels(labels):
    U = np.zeros((len(labels),len(set(labels))))
    
    for node,label in zip(range(U.shape[0]),labels):
        U[node,label] = 1
        
    return U @ U.T,U

In [24]:
def get_labels_from_k_means(k, coords):
    
    kmeans = KMeans(n_clusters=k, random_state=0).fit(coords)
    
    return kmeans.labels_

In [25]:
k = max(graph_meta.sizes)
all_coords = get_all_coords(graph_meta.list_graphs)
kmeans_labels = get_labels_from_k_means(k, all_coords)

/home/rohit/anaconda3/envs/Slam_python_3.8/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [26]:
res_kmeans,U_kmeans = create_perm_from_labels(kmeans_labels)

In [27]:
f1_kmeans, prec_kmeans, rec_kmeans = matching.compute_f1score(res_kmeans,res)
f1_kmeans, prec_kmeans, rec_kmeans

(0.8338782500389226, 0.721055465805062, 0.9885566629752677)

In [46]:
np.arange(-5, 5, 10/20)

array([-5. , -4.5, -4. , -3.5, -3. , -2.5, -2. , -1.5, -1. , -0.5,  0. ,
        0.5,  1. ,  1.5,  2. ,  2.5,  3. ,  3.5,  4. ,  4.5])

In [43]:
np.linspace(0,1,11)

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ])